In [1]:
import os

import altair as alt
import numpy as np
import pandas as pd
import pydeck as pdk
import streamlit as st
import datetime
import plotly.express as px
import plotly.graph_objects as go
from persiantools.jdatetime import JalaliDate

In [35]:
@st.cache_resource
def load_data():
    path = "data/data.csv.zip"
    if not os.path.isfile(path):
        path = "https://github.com/amirsoleix/traffic-counter-app/blob/master/data/data.csv.zip"

    data = pd.read_csv(
        path,
        names = [
            'road code', 'road name', 'start time', 'end time', 'operation length (minutes)', 'class 1', 'class 2',
            'class 3', 'class 4', 'class 5', 'estimated number', 'province', 'start city', 'end city', 'edge name'
        ],
        skiprows=1,
        usecols=[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 15, 16, 17, 18, 19]
    )
    data['start date'] = data['start time'].apply(lambda x: JalaliDate(
        int(x.split(' ')[0].split('/')[0]),
        int(x.split(' ')[0].split('/')[1]),
        int(x.split(' ')[0].split('/')[2])
    ))
    data['end date'] = data['end time'].apply(lambda x: JalaliDate(
        int(x.split(' ')[0].split('/')[0]),
        int(x.split(' ')[0].split('/')[1]),
        int(x.split(' ')[0].split('/')[2])
    ))

    return data

@st.cache_resource
def load_coordinates():
    path = "data/coordinates.csv.zip"
    if not os.path.isfile(path):
        path = "https://github.com/amirsoleix/traffic-counter-app/blob/master/data/coordinates.csv.zip"

    coordinates = pd.read_csv(
        path,
        names=[
            "city",
            "lat",
            "lon"
        ],
        skiprows=1,
        usecols=[0, 1, 2]
    )

    return coordinates

@st.cache_resource
def load_population():
    path = "data/population.csv.zip"
    if not os.path.isfile(path):
        path = "https://github.com/amirsoleix/traffic-counter-app/blob/master/data/population.csv.zip"
    
    population = pd.read_csv(
        path,
        names=[
            "city",
            "population"
        ],
        skiprows=1,
        usecols=[0, 1]
    )

    return population

@st.cache_resource
def counter_years(data):
    years = [
        [JalaliDate(1395, 12, 20), JalaliDate(1396, 1, 18)],
        [JalaliDate(1396, 12, 20), JalaliDate(1397, 1, 18)],
        [JalaliDate(1397, 12, 20), JalaliDate(1398, 1, 18)],
        [JalaliDate(1398, 12, 20), JalaliDate(1399, 1, 18)],
        [JalaliDate(1399, 12, 20), JalaliDate(1400, 1, 18)],
        [JalaliDate(1400, 12, 20), JalaliDate(1401, 1, 18)],
        [JalaliDate(1401, 12, 20), JalaliDate(1402, 1, 18)],
    ]

    # Create a df with columns for year, Gilan, Mazandaran, Golestan
    df = pd.DataFrame(columns=["year", "Rasht", "Sari", "Gorgan"])

    # For each year, calculate the number of cars in each province
    cities = ['Rasht', 'Sari', 'Gorgan']
    count = [0, 0, 0]
    for year in years:
        counter = 0
        for city in cities:
            count[counter] = int(data[(data['start date'] >= year[0]) & (data['end date'] <= year[1]) & (data['end city'] == city)]['class 1'].sum()) + int(data[(data['start date'] >= year[0]) & (data['end date'] <= year[1]) & (data['end city'] == city)]['class 2'].sum())
            counter += 1
        df = pd.concat(
            [df, pd.DataFrame([[year[0].year, count[0], count[1], count[2]]], columns=["year", "Rasht", "Sari", "Gorgan"])],
        )

    return df

In [36]:
data = load_data()
years = counter_years(data)

In [37]:
years

,year,Rasht,Sari,Gorgan
0,1395,2001012,3382710,1306184
0,1396,3053072,3862194,1867153
0,1397,3018392,3502087,1498473
0,1398,1456302,1857193,851570
0,1399,2753519,3054074,1548262
0,1400,3101943,3501340,2320271
0,1401,3716040,3247088,976349
